## The realm of possibility on VarageSale is more like MineCraft than Airbnb.

#### TODO:

- adapt sql for other network types
- are inweight and outwieght useful?

#### Edge Types

- Transaction
- Message
- Praise
- Comment
- Interest

In [180]:
# initializing

import pandas as pd
import pandas_gbq as pgbq
from multiprocessing import Pool
from scipy import stats
import os
import time
import numpy as np

directory = '/Users/joshharris/community_health/sql/cumulative_networks/transaction'
sql_modules = list(map(lambda x: os.path.splitext(x)[0], os.listdir(network_directory)))
google_project_id = 'solid-ridge-104914'
dataset = 'community_networks'

In [181]:
# worker function

def bigquery_worker(i):
    sql_module_type = os.path.basename(directory)
    sql_module = sql_modules[i]
    print('Starting module ',i,': ',sql_modules[i], sep='')
    try:
        result = pgbq.read_gbq('select * from community_networks.'+sql_module_type+'_'+sql_module, google_project_id, dialect='standard', verbose=False)
        print('\n\n',sql_module_type,'_',sql_module,' table found.\n\n', sep='')
    except:
        print('\n\n',sql_module_type,'_',sql_module,' table does not exist, generating now\n\n', sep='')
        with open(directory+'/'+sql_module+'.txt') as query_file:
            query = query_file.read()
        result = pgbq.read_gbq(query, google_project_id, dialect='standard', verbose=False)
        print('\n\nCreating new ',sql_module_type,'_',sql_module,' table\n\n', sep='')
        pgbq.to_gbq(result, dataset+'.'+sql_module_type+'_'+sql_module, google_project_id, if_exists='replace', verbose=False)
        print('\n\nModule ',i,': ',sql_module,'query complete\n\n', sep='')
    return (result)

def add_network_stats(df):
    edges = df['edges']
    nodes = df['nodes']
    avg_indegree = df['avg_indegree']
    avg_outdegree = df['avg_outdegree']
    df['network_density_X_100'] = (edges/(nodes*(nodes-1)/2))*100
    df['avg_degree'] = 2*edges/nodes
    df['indegree_skew'] = (avg_indegree-avg_outdegree)/avg_outdegree
    return(df)
        

In [186]:
#Run network sql modules and join with community facts

if __name__ == '__main__':
    with Pool(processes=len(sql_modules)) as pool:
        result = pool.map(bigquery_worker, (range(len(sql_modules),)))
        pool.close()
        pool.join()


joined = pgbq.read_gbq('select * from '+dataset+'.community_facts_main', google_project_id, dialect='standard', verbose=False)
print('\n\nJoining ',network_type,' modules to community facts\n\n', sep='')
for i in range(len(result)):
    joined = joined.join(result[i].set_index('cid'), how='left', lsuffix='_left')
transaction_main = add_network_stats(joined)
transaction_main.drop(transaction_main[transaction_main.nodes < 0].index, inplace=True)
transaction_main.replace([np.inf,-np.inf], np.nan)
transaction_main.fillna(0, inplace=True)
print('\n\nDone\n\n', sep='')


Starting module 1: edges
Starting module 0: edge_weight
Starting module 2: indegree
Starting module 3: outdegree


transaction_edges table does not exist, generating now




transaction_edge_weight table does not exist, generating now




transaction_indegree table found.




transaction_outdegree table found.




Creating new transaction_edges table




Module 1: edgesquery complete




Creating new transaction_edge_weight table




Module 0: edge_weightquery complete




Joining transaction modules to community facts




Done




In [184]:
len(transaction_main)

2744

In [187]:
# Cross correlate community facts and output key correlations

variables = list(transaction_main)[2:len(transaction_main)] #this is not a good way to create the list of variables.. why not

transaction_corr_matrix = pd.DataFrame(index=variables, columns=variables)
transaction_key_correlations = pd.DataFrame(columns=['pair','coefficient'])
corr_checked = list()

i = 0
j = 0

for i in range(len(variables)):
    for j in range(len(variables)):
        pair = variables[i]+','+variables[j]
        rho, pval = stats.spearmanr(transaction_main[[variables[i],variables[j]]])
        transaction_corr_matrix.set_value(variables[i], variables[j], rho)
        if pair not in corr_checked and rho < 0.99 and (rho > 0.5 or rho < -0.5):
            transaction_key_correlations.loc[len(transaction_key_correlations)] = [pair, rho]
            corr_checked.append(variables[j]+','+variables[i])

transaction_corr_matrix.reset_index(inplace=True)
transaction_key_correlations.sort_values(by='coefficient', ascending=False, inplace=True)
transaction_key_correlations.reset_index(drop=True, inplace=True)

#Save to BigQuery
#pgbq.to_gbq(transaction_corr_matrix, 'community_networks.transaction_corr_matrix', google_project_id, if_exists='replace')
#pgbq.to_gbq(transaction_key_correlations, 'community_networks.transaction_key_correlations', google_project_id, if_exists='replace')
transaction_key_correlations

,pair,coefficient
0,"MAU,mau_june_2017",0.943243
1,"network_density_X_100,avg_degree",0.941987
2,"MAU,total_signups_june",0.933421
3,"nodes,mau_june_2017",0.929237
4,"MAU,new_listings",0.907879
5,"MAU,nodes",0.896438
6,"nodes,new_listings",0.890751
7,"mau_june_2017,new_listings",0.887714
8,"nodes,inventory",0.881711
9,"mau_june_2017,total_signups_june",0.864547


In [188]:
transaction_corr_matrix

,index,age,m1_total_retention,m3_total_retention,MAU,nodes,items_sold_june_2017,items_sold_change_2017,inventory,mau_june_2017,...,total_signups_june,w4_paid_retention,w4_total_retention,avg_weight,edges,avg_indegree,avg_outdegree,network_density_X_100,avg_degree,indegree_skew
0,age,1,0.180922,0.210814,0.232404,0.482618,0.290941,-0.0508378,0.576068,0.26918,...,0.0654493,-0.280539,-0.0349069,-0.00582638,0.0164431,0.00789482,0.00447922,-0.396162,-0.282772,0.00110655
1,m1_total_retention,0.180922,1,0.79734,0.706714,0.679346,0.686391,0.139372,0.595012,0.713792,...,0.698689,0.24079,0.551412,-0.0137173,0.0145708,0.0220197,-0.00846262,-0.506447,-0.353102,0.0347757
2,m3_total_retention,0.210814,0.79734,1,0.701031,0.673172,0.692888,0.148666,0.608388,0.702423,...,0.682783,0.241929,0.536152,-0.00626672,-0.010294,0.00633599,-0.0187307,-0.521715,-0.373107,0.0323877
3,MAU,0.232404,0.706714,0.701031,1,0.896438,0.850928,0.156102,0.705421,0.943243,...,0.933421,0.387402,0.715715,-0.0359565,0.00802741,0.000920091,-0.0198501,-0.685864,-0.484904,0.0113368
4,nodes,0.482618,0.679346,0.673172,0.896438,1,0.828901,0.0861478,0.881711,0.929237,...,0.77267,0.183812,0.566463,-0.032579,0.00690163,-0.00260815,-0.0202687,-0.761531,-0.539558,0.0084772
5,items_sold_june_2017,0.290941,0.686391,0.692888,0.850928,0.828901,1,0.288995,0.770016,0.853794,...,0.81836,0.270869,0.641791,-0.0280951,-0.00256029,0.00670381,-0.0284179,-0.64118,-0.453599,0.0303417
6,items_sold_change_2017,-0.0508378,0.139372,0.148666,0.156102,0.0861478,0.288995,1,0.050939,0.149304,...,0.204368,0.198098,0.182962,-0.00276391,-0.0106054,0.0024903,-0.0181689,-0.0739608,-0.0532804,0.0217981
7,inventory,0.576068,0.595012,0.608388,0.705421,0.881711,0.770016,0.050939,1,0.772752,...,0.610523,0.0486714,0.430248,-0.00916197,0.00283775,0.00615964,-0.0129039,-0.679706,-0.484805,0.0192258
8,mau_june_2017,0.26918,0.713792,0.702423,0.943243,0.929237,0.853794,0.149304,0.772752,1,...,0.864547,0.368167,0.696867,-0.0327624,-7.64666e-05,-0.00144246,-0.0248782,-0.715124,-0.511525,0.01659
9,mau_change_2017,-0.39021,0.338789,0.301387,0.471698,0.188028,0.335621,0.23898,-0.0310017,0.433984,...,0.569942,0.530432,0.540005,-0.0169324,-0.00600695,-0.00290811,-0.0137751,-0.129351,-0.0822343,0.00696375


In [178]:
transaction_main.columns.values

array(['cid', 'name', 'age', 'm1_total_retention', 'm3_total_retention',
       'MAU', 'nodes', 'items_sold_june_2017', 'items_sold_change_2017',
       'inventory', 'mau_june_2017', 'mau_change_2017', 'new_listings',
       'ntile', 'paid_signups_june', 'total_signups_june',
       'w4_paid_retention', 'w4_total_retention', 'avg_weight', 'edges',
       'avg_indegree', 'avg_outdegree', 'network_density_X_100',
       'avg_degree', 'indegree_skew'], dtype=object)

In [189]:
avgdeg = pd.DataFrame(transaction_main[['cid','edges','avg_weight','avg_degree','network_density_X_100']])
avgdeg.sort_values(by='cid', inplace=True)
avgdeg

,cid,edges,avg_weight,avg_degree,network_density_X_100
2625,1,113.0,1.380531,0.005404,1.292263e-05
667,2,373.0,1.319035,1.942708,5.072346e-01
2229,3,3.0,3.333333,0.000087,1.275072e-07
144,4,77638.0,1.803923,5.497274,1.946282e-02
316,5,30798.0,1.483311,3.968814,2.557391e-02
377,6,8680.0,2.036060,32.267658,6.008875e+00
2057,7,505.0,1.910891,0.339610,1.142314e-02
1304,8,162.0,1.456790,0.045107,6.280493e-04
791,9,409.0,1.669927,11.685714,1.693582e+01
2448,10,280.0,1.650000,50.909091,5.090909e+02


### Notes on Correlations among transaction network properties
 
OBSERVATIONS
 
 Inventory correlations:
 
 - Communities where the average seller has a high number of buyers are also likely to have more inventory (0.658)
 - density is inversly correlated with inventory (row 17, -0.596)